In [ ]:
import os
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16


plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc("lines", linewidth=4)
plt.rc("grid", linestyle="--", color="black", alpha=0.5)
plt.rcParams["font.family"] = "serif"

colors_agents = {
    0: "#f9f06b",
    1: "#cdaB8f",
    2: "#dc8add",
    3: "#99c1f1",
    "agg-no-sec": "#5f0d0d",
    "agg-sec": "#24cd1f",
}

colors_trust_metric = {
    "agent": "#051b99",
    "track": "#6a7806",
}


colors_tracks = list(mcolors.TABLEAU_COLORS.keys())

fig_out_dir = os.path.join("figures", "case_study_metrics")
os.makedirs(fig_out_dir, exist_ok=True)

In [ ]:
def get_metrics_axes():
    # Initialize figure and axes and save to class
    fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey="row")

    # left column is for baseline, right for attacked
    # top row for OSPA, middle for F1-score, bottom trust metric
    loop_tuples = [
        ("{}", [0, 1], 1),
        ("OSPA Metric", [0, 10], 0),
        # ("Trust Metric", [0, 1], 1),
    ]
    for i_row, (ylabel, ylim, goal) in enumerate(loop_tuples):
        for j_col, case in enumerate(["Benign", "Attacked"]):
            ax = axs[i_row, j_col]
            title_txt = f"{case}, {ylabel} (Goal: {goal})"
            ax.set_title(title_txt)
            ax.set_xlabel("Time (s)")
            ax.set_ylabel(ylabel)
            ax.set_ylim(ylim)
            ax.grid()
    return fig, axs


def get_trust_metrics_axes():
    fig, axs = plt.subplots(1, 2, figsize=(8, 2.5), sharey=True)
    ylim = [0, 1]
    goal = 1
    for j_col, case in enumerate(["Benign", "Attacked"]):
        ax = axs[j_col]
        title_txt = f"{case}, (Goal: {goal})"
        ax.set_title(title_txt)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Trust Metric")
        ax.set_ylim(ylim)
        ax.grid()
    return fig, axs

In [ ]:
from avstack_rosbag import DatasetPostprocessor


def load_postprocs(case_idx):
    case_idx_to_scene = {0: 0, 1: 1, 2: 1}
    postproc_benign = DatasetPostprocessor(
        bag_folder="/data/shared/CARLA/rosbags/baseline_intersection/",
        bag_name=f"baseline_intersection_{case_idx_to_scene[case_idx]}",
    )
    print(f"Loaded postprocessor of length {len(postproc_benign)}")

    postproc_attack = DatasetPostprocessor(
        bag_folder="/data/shared/CARLA/rosbags/cte_case/",
        bag_name=f"cte_case_{case_idx}_attacked",
    )
    print(f"Loaded postprocessor of length {len(postproc_attack)}")

    return postproc_benign, postproc_attack

In [ ]:
from avstack_bridge import MetricsBridge
from avstack_bridge import ObjectStateBridge, TrackBridge
from avstack.metrics import get_instantaneous_metrics
from avtrust_bridge import TrustBridge
from avtrust.metrics import get_trust_agents_metrics, get_trust_tracks_metrics


def make_metrics_figures(
    postproc_benign, postproc_attack, case_idx, t_max, assign_metric
):
    fig, axs = get_metrics_axes()
    fig_trust, axs_trust = get_trust_metrics_axes()

    linewidth_small = 2
    linewidth_large = 3

    metrics_all = {
        "benign": {
            "no-sec": {
                "ospa": None,
                "f1": None,
                "time": None,
            },
            "sec": {
                "ospa": None,
                "f1": None,
                "agent_trust": None,
                "track_trust": None,
                "time": None,
            },
        },
        "attacked": {
            "no-sec": {
                "ospa": None,
                "f1": None,
                "time": None,
            },
            "sec": {
                "ospa": None,
                "f1": None,
                "agent_trust": None,
                "track_trust": None,
                "time": None,
            },
        },
    }

    case_dir = os.path.join(fig_out_dir, f"case{case_idx}")
    os.makedirs(case_dir, exist_ok=True)

    for i_col, postproc in enumerate([postproc_benign, postproc_attack]):
        ################################
        # PLOT ASSIGNMENT/OSPA METRICS
        ################################
        # -------------------
        # All agents
        # -------------------
        for ID_agent in range(4):
            for sensor in ["lidar0"]:
                for level in ["detections"]:
                    # get the metrics data for this agent/sensor/level
                    topic = f"/metrics/agent{ID_agent}/{sensor}/{level}"
                    metrics_msgs = postproc._get_messages_by_time(topic)
                    metrics_data = {
                        k: MetricsBridge.assignment_metrics_ros_to_avstack(v)
                        for k, v in metrics_msgs["data"].items()
                    }
                    times = np.array(list(metrics_data.keys()))
                    ospas = np.array([met.ospa for met in metrics_data.values()])
                    f1s = np.array([met.f1_score for met in metrics_data.values()])
                    precs = np.array([met.precision for met in metrics_data.values()])
                    recalls = np.array([met.recall for met in metrics_data.values()])
                    idx_times = times <= t_max

                    # f1 score for agents in top row
                    if assign_metric == "precision":
                        assign_metric_plot = precs
                        assign_title = "Precision"
                    elif assign_metric == "recall":
                        assign_metric_plot = recalls
                        assign_title = "Recall"
                    elif assign_metric == "f1-score":
                        assign_metric_plot = f1s
                        assign_title = "F1-Score"
                    else:
                        raise NotImplementedError(assign_metric)
                    axs[0, i_col].plot(
                        times[idx_times],
                        assign_metric_plot[idx_times],
                        color=colors_agents[ID_agent],
                        label=f"Agent {ID_agent}",
                        linewidth=linewidth_small,
                    )
                    axs[0, i_col].set_title(
                        axs[0, i_col].get_title().format(assign_title)
                    )
                    axs[0, i_col].set_ylabel(
                        axs[0, i_col].get_ylabel().format(assign_title)
                    )

                    # ospa for agents in middle row
                    axs[1, i_col].plot(
                        times[idx_times],
                        ospas[idx_times],
                        color=colors_agents[ID_agent],
                        label=f"Agent {ID_agent}",
                        linewidth=linewidth_small,
                    )

        # -------------------
        # Aggregator
        # -------------------
        loop_tuples = [
            ("Aggregator, No Security", "agg-no-sec", "/metrics/fusion"),
            (
                "Aggregator, Security-Aware",
                "agg-sec",
                "/metrics/security_aware_fusion/assignment",
            ),
        ]
        for label, agg_name, topic in loop_tuples:
            # load the metrics for the agg
            metrics_msgs_agg = postproc._get_messages_by_time(topic)
            metrics_data_agg = {
                k: MetricsBridge.assignment_metrics_ros_to_avstack(v)
                for k, v in metrics_msgs_agg["data"].items()
            }
            times_agg = np.array(list(metrics_data_agg.keys()))
            ospas_agg = np.array([met.ospa for met in metrics_data_agg.values()])
            f1s_agg = np.array([met.f1_score for met in metrics_data_agg.values()])
            idx_times = times_agg <= t_max

            # add the metrics for the aggregator
            alg = "no-sec" if "no" in label.lower() else "sec"
            case = "benign" if i_col == 0 else "attacked"
            metrics_all[case][alg]["ospa"] = ospas_agg[idx_times]
            metrics_all[case][alg]["f1"] = f1s_agg[idx_times]
            metrics_all[case][alg]["time"] = times_agg[idx_times]

            # add f1 score to plot
            axs[0, i_col].plot(
                times_agg[idx_times],
                f1s_agg[idx_times],
                color=colors_agents[agg_name],
                linewidth=linewidth_large,
                label=label,
            )

            # add ospa to plot
            axs[1, i_col].plot(
                times_agg[idx_times],
                ospas_agg[idx_times],
                color=colors_agents[agg_name],
                linewidth=linewidth_large,
                label=label,
            )

        ################################
        # PLOT TRUST METRICS
        ################################
        trust_loops = [
            ("agent", TrustBridge.agent_trust_metric_array_ros_to_avstack),
            ("track", TrustBridge.track_trust_metric_array_ros_to_avstack),
        ]
        for topic_fill, trust_metric_bridge in trust_loops:
            # get the data
            topic = f"/metrics/security_aware_fusion/{topic_fill}_trust"
            metrics_trust_msgs = postproc._get_messages_by_time(topic)
            metrics_trust_data = {
                k: trust_metric_bridge(v) for k, v in metrics_trust_msgs["data"].items()
            }
            times_trust = np.array(list(metrics_trust_data.keys()))
            metric_means = np.array(
                [met.mean_metric for met in metrics_trust_data.values()]
            )
            idx_times = times_trust <= t_max

            # add the metrics for the aggregator
            case = "benign" if i_col == 0 else "attacked"
            alg = "sec"
            metrics_all[case][alg][f"{topic_fill}_trust"] = metric_means

            # add trust to plot
            axs_trust[i_col].plot(
                times_trust[idx_times],
                metric_means[idx_times],
                color=colors_trust_metric[topic_fill],
                linewidth=linewidth_large,
                label=f"{topic_fill.title()}",
            )

    # add the legend for the assignment/ospas
    legitems = [
        *["Agent {}".format(i) for i in range(4)],
        "Aggregator, No Security",
        "Aggregator, Security-Aware",
    ]
    leg = fig.legend(
        legitems,
        loc="lower center",
        bbox_to_anchor=(0.5, -0.15),
        ncol=2,
        fancybox=True,
        shadow=True,
        prop={"family": "serif"},
    )
    axs_trust[1].legend(
        ["Agent Trust", "Track Trust"],
        loc="lower right",
        prop={"family": "serif"},
    )

    # # show the plot
    fig.tight_layout()
    fig_trust.tight_layout()

    # save metrics figure
    fig.savefig(
        os.path.join(case_dir, f"metrics_ospa_plot_{case_idx:03d}.pdf"),
        bbox_extra_artists=(leg,),
        bbox_inches="tight",
    )
    fig.savefig(
        os.path.join(case_dir, f"metrics_ospa_plot_{case_idx:03d}.png"),
        bbox_extra_artists=(leg,),
        bbox_inches="tight",
    )

    # save trust metrics figure
    fig_trust.savefig(os.path.join(case_dir, f"metrics_trust_plot_{case_idx:03d}.pdf"))
    fig_trust.savefig(os.path.join(case_dir, f"metrics_trust_plot_{case_idx:03d}.png"))

    plt.show()

    return metrics_all

In [ ]:
def print_metric_summary_stats(metrics):
    # metric reduction compared to baseline
    for metric in ["ospa"]:
        t_eval = metrics["benign"]["no-sec"]["time"]
        sec_att_interp = np.interp(
            t_eval,
            metrics["attacked"]["sec"]["time"],
            metrics["attacked"]["sec"][metric],
        )
        nosec_att_interp = np.interp(
            t_eval,
            metrics["attacked"]["no-sec"]["time"],
            metrics["attacked"]["no-sec"][metric],
        )
        benign_result = metrics["benign"]["no-sec"][metric]
        nosec_error_vs_benign = np.maximum(0, nosec_att_interp - benign_result)
        sec_error_vs_benign = np.maximum(0, sec_att_interp - benign_result)
        sec_error_reduction = (
            nosec_error_vs_benign - sec_error_vs_benign
        ) / nosec_error_vs_benign
        sec_error_reduction = sec_error_reduction[~np.isinf(sec_error_reduction)]
        print(
            f"Mean error reduction for {metric}: {np.nanmean(sec_error_reduction):.4f}"
        )

    # average trust accuracy
    for case in metrics:
        for target in ["agent_trust", "track_trust"]:
            print(
                f"Average {target} metric for {case}: {np.mean(metrics[case]['sec'][target]):.4f}"
            )

In [ ]:
metric_to_plot = {0: "precision", 1: "precision", 2: "recall"}
for i_case in [1]:  # range(3):
    postproc_benign, postproc_attack = load_postprocs(i_case)
    metrics_all = make_metrics_figures(
        postproc_benign,
        postproc_attack,
        i_case,
        t_max=10,
        assign_metric=metric_to_plot[i_case],
    )
    print_metric_summary_stats(metrics_all)